In [1]:
import sys
!{sys.executable} -m pip install pandas sdv table_evaluator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.7/246.7 kB 7.4 MB/s eta 0:00:00
  Attempting uninstall: psutil
    Found existing installation: psutil 5.9.0
    Uninstalling psutil-5.9.0:
      Successfully uninstalled psutil-5.9.0


In [4]:
# Importieren der benötigten Bibliotheken

import pandas as pd
import sdv
from sdv.single_table import CTGANSynthesizer
from sdv.metadata import SingleTableMetadata
from sdv.evaluation.single_table import evaluate_quality
from table_evaluator import load_data, TableEvaluator

In [5]:
# Einspielen des Datensatzes
kadeco=pd.read_csv("kadeco_cleaned.csv")
data= kadeco
#Reduzieren des Datensatzes auf die nötigen Features
data=data[['deliverer', 'country', 'size','weight', 'amount', 'product', 'is_damaged']]
data=pd.get_dummies(data)# One Hot Encoding des Datensatzes mit einer Methode aus der Pandas Bibliothek

In [6]:
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(data=data)


In [11]:
synthesizer = CTGANSynthesizer(metadata
                               #,epochs=300
                               )

In [8]:
synthesizer.fit(data)
synthesizer.save("ctgan_augmenter.pkl")


KeyboardInterrupt: 

In [ ]:
# Generieren von augmentierten Daten
synthetic_data = synthesizer.sample(
    num_rows=10000
    #,batch_size=10
)
synthetic_data.to_csv("synthetic_data.csv")

In [ ]:
# Evaluierung der augmentierten Daten
print(data.shape, synthetic_data.shape)
table_evaluator =  TableEvaluator(data, synthetic_data)
table_evaluator.evaluate(target_col="is_damaged")

In [ ]:
#Visualierung
table_evaluator.visual_evaluation()